# Install and Import property

In [1]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Making key point using Open CV

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
#mp_holistic??

In [7]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# Extract keypoint

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Setup Folders for Collection

In [9]:
# Path for exported data, numpy arrays

DATA_PATH = os.path.join('C:\\Users\\alpian\\taSignLanguage','MP_Data')

# Actions that we try to detect
actions = np.array(['apa', 'dimana', 'kapan'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

# Collect Keypoint Values for Training and Testing

# Preprocess Data and Create Labels and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'apa': 0, 'dimana': 1, 'kapan': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
np.array(sequences).shape

(90, 30, 1662)

In [15]:
X = np.array(sequences)

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Build and Train LSTM Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [21]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, epochs=250, callbacks=[tb_callback], validation_data=(X_test,y_test))

Epoch 1/250
3/3 [==============================] - 6s 449ms/step - loss: 1.7484 - accuracy: 0.2778 - val_loss: 3.7264 - val_accuracy: 0.5000
Epoch 2/250
3/3 [==============================] - 0s 87ms/step - loss: 5.9427 - accuracy: 0.4444 - val_loss: 5.6937 - val_accuracy: 0.5000
Epoch 3/250
3/3 [==============================] - 0s 87ms/step - loss: 4.2837 - accuracy: 0.2917 - val_loss: 1.1073 - val_accuracy: 0.5000
Epoch 4/250
3/3 [==============================] - 0s 87ms/step - loss: 1.4735 - accuracy: 0.2917 - val_loss: 1.6054 - val_accuracy: 0.2778
Epoch 5/250
3/3 [==============================] - 0s 87ms/step - loss: 1.4542 - accuracy: 0.3194 - val_loss: 1.1947 - val_accuracy: 0.1667
Epoch 6/250
3/3 [==============================] - 0s 88ms/step - loss: 1.1227 - accuracy: 0.3611 - val_loss: 1.1293 - val_accuracy: 0.1667
Epoch 7/250
3/3 [==============================] - 0s 87ms/step - loss: 1.0830 - accuracy: 0.3750 - val_loss: 1.0922 - val_accuracy: 0.1667
Epoch 8/250
3/3 [==

3/3 [==============================] - 0s 93ms/step - loss: 0.2550 - accuracy: 0.9444 - val_loss: 0.4235 - val_accuracy: 0.7778
Epoch 60/250
3/3 [==============================] - 0s 96ms/step - loss: 0.2346 - accuracy: 0.9167 - val_loss: 0.4762 - val_accuracy: 0.7778
Epoch 61/250
3/3 [==============================] - 0s 97ms/step - loss: 0.2120 - accuracy: 0.9167 - val_loss: 0.2179 - val_accuracy: 1.0000
Epoch 62/250
3/3 [==============================] - 0s 105ms/step - loss: 0.1603 - accuracy: 0.9444 - val_loss: 0.1326 - val_accuracy: 0.9444
Epoch 63/250
3/3 [==============================] - 0s 89ms/step - loss: 1.7602 - accuracy: 0.7778 - val_loss: 9.3046 - val_accuracy: 0.1667
Epoch 64/250
3/3 [==============================] - 0s 94ms/step - loss: 5.6867 - accuracy: 0.3889 - val_loss: 0.8791 - val_accuracy: 0.6667
Epoch 65/250
3/3 [==============================] - 0s 88ms/step - loss: 1.2240 - accuracy: 0.5278 - val_loss: 1.7778 - val_accuracy: 0.1667
Epoch 66/250
3/3 [=======

3/3 [==============================] - 0s 89ms/step - loss: 0.4538 - accuracy: 0.7083 - val_loss: 0.5052 - val_accuracy: 0.5000
Epoch 118/250
3/3 [==============================] - 0s 86ms/step - loss: 0.4220 - accuracy: 0.6944 - val_loss: 0.4532 - val_accuracy: 0.5000
Epoch 119/250
3/3 [==============================] - 0s 88ms/step - loss: 0.3671 - accuracy: 0.7917 - val_loss: 0.3847 - val_accuracy: 0.7222
Epoch 120/250
3/3 [==============================] - 0s 86ms/step - loss: 0.3285 - accuracy: 0.7917 - val_loss: 0.2991 - val_accuracy: 0.8333
Epoch 121/250
3/3 [==============================] - 0s 86ms/step - loss: 0.2069 - accuracy: 0.9167 - val_loss: 0.5525 - val_accuracy: 0.8333
Epoch 122/250
3/3 [==============================] - 0s 90ms/step - loss: 0.1612 - accuracy: 0.9444 - val_loss: 0.6635 - val_accuracy: 0.8333
Epoch 123/250
3/3 [==============================] - 0s 88ms/step - loss: 0.1413 - accuracy: 0.9444 - val_loss: 1.3199 - val_accuracy: 0.7778
Epoch 124/250
3/3 [=

Epoch 175/250
3/3 [==============================] - 0s 95ms/step - loss: 0.5155 - accuracy: 0.7083 - val_loss: 0.6699 - val_accuracy: 0.5000
Epoch 176/250
3/3 [==============================] - 0s 87ms/step - loss: 0.4501 - accuracy: 0.7222 - val_loss: 0.4055 - val_accuracy: 0.8333
Epoch 177/250
3/3 [==============================] - 0s 93ms/step - loss: 0.4692 - accuracy: 0.6528 - val_loss: 0.4016 - val_accuracy: 0.8333
Epoch 178/250
3/3 [==============================] - 0s 84ms/step - loss: 0.4517 - accuracy: 0.6806 - val_loss: 0.5459 - val_accuracy: 0.5000
Epoch 179/250
3/3 [==============================] - 0s 87ms/step - loss: 0.4321 - accuracy: 0.7083 - val_loss: 0.6487 - val_accuracy: 0.5000
Epoch 180/250
3/3 [==============================] - 0s 86ms/step - loss: 0.4563 - accuracy: 0.7083 - val_loss: 0.5791 - val_accuracy: 0.5000
Epoch 181/250
3/3 [==============================] - 0s 97ms/step - loss: 0.4304 - accuracy: 0.6944 - val_loss: 0.4009 - val_accuracy: 0.9444
Epoch 

Epoch 233/250
3/3 [==============================] - 0s 92ms/step - loss: 0.4309 - accuracy: 0.7083 - val_loss: 0.5112 - val_accuracy: 0.5000
Epoch 234/250
3/3 [==============================] - 0s 87ms/step - loss: 0.4021 - accuracy: 0.7500 - val_loss: 0.4193 - val_accuracy: 1.0000
Epoch 235/250
3/3 [==============================] - 0s 88ms/step - loss: 0.4116 - accuracy: 0.8333 - val_loss: 0.4040 - val_accuracy: 1.0000
Epoch 236/250
3/3 [==============================] - 0s 89ms/step - loss: 0.4000 - accuracy: 0.8750 - val_loss: 0.5302 - val_accuracy: 0.5000
Epoch 237/250
3/3 [==============================] - 0s 86ms/step - loss: 0.4034 - accuracy: 0.7222 - val_loss: 0.5346 - val_accuracy: 0.5000
Epoch 238/250
3/3 [==============================] - 0s 93ms/step - loss: 0.3931 - accuracy: 0.7500 - val_loss: 0.3630 - val_accuracy: 1.0000
Epoch 239/250
3/3 [==============================] - 0s 86ms/step - loss: 0.4043 - accuracy: 0.7917 - val_loss: 0.6329 - val_accuracy: 0.5000
Epoch 

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Make Predictions

In [24]:
res = model.predict(X_test)

1/1 [==============================] - 1s 603ms/step


In [25]:
actions[np.argmax(res[4])]

'kapan'

In [26]:
actions[np.argmax(y_test[4])]

'dimana'

# Save Weights

In [27]:
model.save('action.h5')

In [28]:
model.load_weights('action.h5')

# Evaluation using Confusion Matrix and Accuracy

In [29]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [30]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 34ms/step


In [31]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [32]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[12,  0],
        [ 0,  6]],

       [[ 9,  0],
        [ 9,  0]],

       [[ 6,  9],
        [ 0,  3]]], dtype=int64)

In [33]:
accuracy_score(ytrue, yhat)

0.5

# Test in Real Time

In [38]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [39]:
cap.release()
cv2.destroyAllWindows()